In [1]:
import numpy as np
import xarray as xr
import dask
import cftime
import os
import intake
import pandas as pd
from collections import defaultdict
from tqdm.autonotebook import tqdm
from xmip.postprocessing import combine_datasets, concat_members,concat_experiments
import gcsfs
from datetime import datetime
fs = gcsfs.GCSFileSystem() #list stores, stripp zarr from filename, load 

/tmp/ipykernel_2936/2254304399.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
def open_zos_ibe_monmeans(zos_path,ibe_path,chunks):
    fns = ['gs://'+k for k in fs.ls(zos_path) if 'DS_Store' not in k]
    zos_monmeans = xr.open_mfdataset(fns,
                                 combine='nested',coords='minimal',compat='override',concat_dim = 'source_id',chunks=chunks,engine='zarr')
    
    fns = ['gs://'+k for k in fs.ls(ibe_path) if 'DS_Store' not in k]
    ibe_monmeans = xr.open_mfdataset(fns,
                                 combine='nested',coords='minimal',compat='override',concat_dim = 'source_id',chunks=chunks,engine='zarr')
    
    zos_ibe_monmeans = xr.merge((zos_monmeans,ibe_monmeans),join='inner')
    zos_ibe_monmeans['zos_ibe'] = zos_ibe_monmeans['zos'] + zos_ibe_monmeans['ibe']

    return zos_ibe_monmeans

def compute_range(ds):
    return ds.max(dim='month') - ds.min(dim='month')

def find_peak_months(ds):
    return 1+ds.argmax(dim='month',skipna=False).where(np.isfinite(ds.isel(month=0,drop=True))), 1+ds.argmin(dim='month',skipna=False).where(np.isfinite(ds.isel(month=0,drop=True)))

Configure the script:

In [3]:
ssps = ['ssp126','ssp245','ssp370','ssp585'] #SSPs to process 

input_path_zos = 'gs://leap-persistent/timh37/CMIP6/zos_psmsl_month_means'
input_path_ibe = 'gs://leap-persistent/timh37/CMIP6/ibe_psmsl_month_means'

'''
input_path_zos = 'gs://leap-persistent/timh37/CMIP6/zos_1x1_month_means'
input_path_ibe = 'gs://leap-persistent/timh37/CMIP6/ibe_1x1_month_means'
'''
output_path = 'gs://leap-persistent/timh37/CMIP6/seasonal_slc/merged'


In [4]:
for s,ssp in tqdm(enumerate(ssps)):
    if 'psmsl' in input_path_zos:
        monmeans = open_zos_ibe_monmeans(input_path_zos, #open gridded data
                                 input_path_ibe,{'tg':1000,'period':2,'member_id':200,'month':12}).sel(experiment_id=ssp)
    else:
        monmeans = open_zos_ibe_monmeans(input_path_zos, #open gridded data
                                 input_path_ibe,{'lat':500,'lon':500,'period':1,'member_id':50,'experiment_id':4,'month':12}).sel(experiment_id=ssp)

    #adjust string dtypes (otherwise zarr complains)
    monmeans['member_id'] = monmeans['member_id'].astype('str') 
    monmeans['source_id'] = monmeans['source_id'].astype('str')
    monmeans['experiment_id'] = monmeans['experiment_id'].astype('str')

    change = monmeans.isel(period=1)-monmeans.isel(period=0)

    imonths = [[-1,0,1],[2,3,4],[5,6,7],[8,9,10],[0,1,2,3,4,5,6,7,8,9,10,11]]
    for s,season in enumerate(['DJF','MAM','JJA','SON','annual']): 
        seas_slc = 100*change.isel(month=imonths[s]).mean(dim='month')
        seas_slc.to_zarr(os.path.join(output_path,season.lower()+'_slc_'+['1x1_','psmsl_']['psmsl' in input_path_zos]+ssp+'.zarr'),mode='w',zarr_format=2)
        seas_slc.mean(dim='member_id').to_zarr(os.path.join(output_path.replace('merged','merged_memmean'),season.lower()+'_slc_'+['1x1_','psmsl_']['psmsl' in input_path_zos]+ssp+'.zarr'),mode='w',zarr_format=2)

0it [00:00, ?it/s]